In [357]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchkeras
from plotly import graph_objects as go
from sklearn.preprocessing import MinMaxScaler

In [358]:
# 导入数据
# 数据下载：https://www.kaggle.com/kankanashukla/champagne-data
df = pd.read_csv('champagne.csv', index_col=0)
df.head()

,Sales
Month,
1964-01,2815
1964-02,2672
1964-03,2755
1964-04,2721
1964-05,2946


In [359]:
# 数据预览
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Sales'], name='Sales'))
fig.show()

In [360]:
# 数据处理
# 标准化
scaler = MinMaxScaler()
predict_field = 'Scaler'
df[predict_field] = scaler.fit_transform(df['Sales'].values.reshape(-1, 1))
df.head()

,Sales,Scaler
Month,,
1964-01,2815,0.112133
1964-02,2672,0.100696
1964-03,2755,0.107334
1964-04,2721,0.104615
1964-05,2946,0.122611


In [361]:
def create_dataset(data: list, time_step: int):
    arr_x, arr_y = [], []
    for i in range(len(data) - time_step - 1):
        x = data[i: i + time_step]
        y = data[i + time_step]
        arr_x.append(x)
        arr_y.append(y)
    return np.array(arr_x), np.array(arr_y)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)

time_step = 8
X, Y = create_dataset(df[predict_field].values, time_step)
# 转化成 tensor->(batch_size, seq_len, feature_size)
X = torch.tensor(X.reshape(-1, time_step, 1), dtype=torch.float).to(device)
Y = torch.tensor(Y.reshape(-1, 1, 1), dtype=torch.float).to(device)
print('Total datasets: ', X.shape, '-->', Y.shape)

# 划分数据
split_ratio = 0.8
len_train = int(X.shape[0] * split_ratio)
X_train, Y_train = X[:len_train, :, :], Y[:len_train, :, :]
print('Train datasets: ', X_train.shape, '-->', Y_train.shape)

# 构建迭代器
batch_size = 5
ds = TensorDataset(X, Y)
dl = DataLoader(ds, batch_size=batch_size, num_workers=0)
ds_train = TensorDataset(X_train, Y_train)
dl_train = DataLoader(ds_train, batch_size=batch_size, num_workers=0)

for x, y in dl_train:
    print(x.shape)
    print(y.shape)
    break



Device: cuda:0
Total datasets:  torch.Size([96, 8, 1]) --> torch.Size([96, 1, 1])
Train datasets:  torch.Size([76, 8, 1]) --> torch.Size([76, 1, 1])
torch.Size([5, 8, 1])
torch.Size([5, 1, 1])


In [362]:
# 定义模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=6, num_layers=3, batch_first=True)
        self.fc = nn.Linear(in_features=6, out_features=1)

    def forward(self, x):
        # print(x)
        # x is input, size (seq_len, batch, input_size)
        x, _ = self.lstm(x)
        print(x.shape)
        # x is output, size (seq_len, batch, hidden_size)
        s, b, h = x.shape
        x = x.reshape(s * b, h)
        x = self.fc(x)
        y = x.view(-1, 1, 1)
        print(y.shape)
        return y

In [363]:
# torchkeras API 训练方式
# model = torchkeras.Model(Net())
# model.summary(input_shape=(time_step, 1))
# model.compile(loss_func=F.mse_loss, optimizer=torch.optim.Adam(model.parameters(), lr=0.01))
# dfhistory = model.fit(epochs=50, dl_train=dl_train, log_step_freq=20)

In [364]:
# 模型评估
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=dfhistory.index, y=dfhistory['loss'], name='loss'))
# fig.show()

In [365]:
# 预测验证预览
# y_pred = model.predict(dl)
# y_pred = y_pred.detach().numpy()
#
# fig = go.Figure()
# fig.add_trace(go.Scatter(y=Y.squeeze(), name='y_true'))
# fig.add_trace(go.Scatter(y=y_pred.squeeze(), name='y_pred'))
# fig.show()

In [366]:
# 自定义训练方式
model = Net().to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)


def train_step(model, features, labels):
    # 正向传播求损失
    predictions = model.forward(features)
    loss = loss_function(predictions, labels)
    # 反向传播求梯度
    loss.backward()
    # 参数更新
    optimizer.step()
    optimizer.zero_grad()
    return loss.item()

# 测试一个batch
# features, labels = next(iter(dl_train))
# loss =  train_step(model, features, labels)
# loss

In [367]:
def train_model(model, epochs):
    for epoch in range(1, epochs + 1):
        list_loss = []
        for features, labels in dl_train:
            lossi = train_step(model, features, labels)
            list_loss.append(lossi)
        loss = np.mean(list_loss)
        if epoch % 10 == 0:
            print('epoch={} | loss={} '.format(epoch, loss))


train_model(model, 20)

torch.Size([5, 8, 6])
torch.Size([40, 1, 1])


d:\python\python379\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning:

Using a target size (torch.Size([5, 1, 1])) that is different to the input size (torch.Size([40, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



RuntimeError: The size of tensor a (40) must match the size of tensor b (5) at non-singleton dimension 0

In [ ]:
# 预测验证预览

y_pred = model.forward(X)
print(X.shape)
print(y_pred.shape)
# print(y_pred)
y_p_pred = y_pred.detach().cpu().numpy().squeeze()
Y_p_true = Y.cpu().squeeze()
fig = go.Figure()
fig.add_trace(go.Scatter(y=Y_p_true, name='y_true'))
fig.add_trace(go.Scatter(y=y_p_pred, name='y_pred'))
fig.show()

In [ ]:
# 外推20个点
# n = 20
#
# list_y_pre = []
# x = X[-1].view(1, 1, time_step)
#
# for n in range(1, n + 1):
#     y = model.forward(x)
#     # y = y.view(-1,1,1)
#     list_y_pre.append(y.item())
#     x = torch.cat([x, y], dim=2)
#     x = x[:, :, 1:]
#
# y_p2_pred = np.concatenate((y_p_pred, np.array(list_y_pre)))
# fig = go.Figure()
# fig.add_trace(go.Scatter(y=Y_p_true, name='y_true'))
# fig.add_trace(go.Scatter(y=y_p2_pred, name='y_pred'))
# fig.show()